### Conection to ESP32 via port ### 

In [1]:
import serial
import time

print("Start")
port="COM5" #This will be different for various devices and on windows it will probably be a COM port.
bluetooth=serial.Serial(port, 115200)#Start communications with the bluetooth unit
print("Connected")
bluetooth.flushInput()

Start
Connected


### Industrial camera settings ###

In [2]:
import numpy as np
import torch
from matplotlib import pyplot as plt
import cv2 as cv
import pypylon.pylon as py
from ultralytics import YOLO

tlf = py.TlFactory.GetInstance()
devices = tlf.EnumerateDevices()
for d in devices:
    print('Name of device: ', d.GetModelName(), '\nSerial number: ', d.GetSerialNumber())

cam = py.InstantCamera(tlf.CreateDevice(devices[0]))
cam = py.InstantCamera(tlf.CreateFirstDevice())
cam.Open()
cam.TriggerSelector = "FrameStart"
# selector handling:
# first set selector than the dependent value:
cam.TriggerSelector = "FrameStart"
cam.TriggerSource = "Line1"
cam.PixelFormat = "Mono8"
# like above, alternative is the long form
cam.PixelFormat.SetValue("Mono8")
res = cam.GrabOne(200)
# full method call
img = res.GetArray()
# abbrev
img = res.Array
img = img/255

c:\users\admin\appdata\local\programs\python\python38\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Name of device:  acA1920-40gmDVL 
Serial number:  22193660


### Connecting the YOLOv8 model and processing each frame in a loop ###

In [33]:
import numpy as np

consecutive_frame = 3
frame_count = 0

model = YOLO("C:/Users/admin/Downloads/Basler Camera Test/yolov8/best_own_dataset.pt")
#connection to computer GPU
model = model.to('cuda')
collect = []
pred = []
start_collect = time.time()
while(True):
    res = cam.GrabOne(500)
    img = res.GetArray()
    img = res.Array
    
    #cropping the image to a square
    img = cv.cvtColor(img, cv.COLOR_GRAY2RGB)[800:1300, 200:1200]
    
    torch_img = torch.from_numpy(img).device
    
    #time to get a frame from an industrial camera
    finish_collect = time.time()
    collect_time = finish_collect - start_collect
    collect.append(collect_time)
    
    #time for predictioni of one frame 
    start_pred = time.time()
    prediction_result = model.predict(source = img, conf=0.25, show=True )
    finish_pred = time.time()
    pred_time = finish_pred - start_pred 
    
    #record information to array 
    pred.append(pred_time)
    boxes = []
    labels = []

    for i in prediction_result[0]:
        i = i.cpu().numpy()
        boxes.append(np.append(i.boxes.xyxy[0], i.boxes.cls[0]))

    out = 0 
    LGD = np.zeros(7)
    for (xA, yA, xB, yB, cls) in boxes:
        n = int((yA+yB/2) // 250)
        if ((n < 5) and (LGD[n] == 0) and (xA+xB/2) > 250):
            LGD[n] = int(1)
            t=int(5+cls)
            LGD[t] = int(1)
    for k in np.arange(7):
        out = out + LGD[k]*2**k
    out = int(out)
    ##if outprev != out : 
    bluetooth.write(str.encode(str(int(out)) + "\n"))#These need to be bytes not unicode

    if cv.waitKey(1) & 0xFF == ord('q'):
        break

# finally, close the window
cv.destroyAllWindows()
cv.waitKey(1)

Ultralytics YOLOv8.0.20  Python-3.8.9 torch-1.13.1+cpu CPU
Model summary (fused): 168 layers, 3006038 parameters, 0 gradients, 8.1 GFLOPs


-1

### Getting the coordinates of which section the trash is in ###

In [10]:
for (xA, yA, xB, yB, cls) in boxes:
    print(xA, yA, xB, yB, cls)

485.0 625.0 866.0 1066.0 1.0
483.0 656.0 728.0 1065.0 1.0
1454.0 1.0 1920.0 428.0 1.0
1296.0 184.0 1543.0 516.0 0.0


### Create cvs file to store information about time r

In [ ]:
#choose the speed
import pandas as pd
speed_type = "slow" #slow,middle,fast

file = {'speed':speed_type, 'Collection': collect, 'Analysis and Display': pred}
df = pd.DataFrame(file)
pd.DataFrame.to_csv(df, 'time_info_speed.csv')

In [ ]:
pd.read_csv('file_veryquick.csv')